In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'

In [1]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Set the required configuration
spark_conf = pyspark.SparkConf()
# spark_conf.set("spark.sql.warehouse.dir", "/path/to/spark/warehouse")
# spark_conf.set("hive.metastore.uris", "thrift://hive-metastore:9083")
# spark_conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark_conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,org.apache.spark:spark-hadoop-cloud_2.13:3.3.1,org.apache.hadoop:hadoop-common:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.172")
# spark_conf.set("spark.jars.packages", "org.apache.spark:spark-hadoop-cloud_2.13")

def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")
    spark_context._jsc.hadoopConfiguration().set("mapreduce.outputcommitter.factory.scheme.s3a",
                                                 "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.committer.name", "partitioned")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.committer.staging.conflict-mode", "fail")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.committer.staging.tmp.path", "tmp/partitioned")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.buffer.dir", "/tmp")


# Create the SparkSession with the configuration and enable Hive support
spark = (
    SparkSession.builder
    .master("local[*]")
    .config(conf=spark_conf)
    .enableHiveSupport()
    .getOrCreate()
)

load_config(spark.sparkContext)

In [2]:
from pyspark.sql.types import StructType, StructField, LongType, TimestampType, DoubleType, StringType

schema = StructType([
    StructField('VendorID', LongType(), True),
    StructField('tpep_pickup_datetime', StringType(), True),
    StructField('tpep_dropoff_datetime', StringType(), True),
    StructField('passenger_count', DoubleType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('RatecodeID', DoubleType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('PULocationID', LongType(), True),
    StructField('DOLocationID', LongType(), True),
    StructField('payment_type', LongType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True)])

df = spark.read.option("header", "true").schema(schema).csv("s3a://warehouse/taxi-data.csv")

## Write in Parquet with PULocationID as Partition

In [3]:
df.write.partitionBy("PULocationID").format("parquet").save("s3a://warehouse/partition/nyc_taxis/data.parquet")

## in Parquet with PULocationID as Partition

Minics delete and recreated the files

In [4]:
df.write.mode("overwrite").partitionBy("PULocationID").format("parquet").save("s3a://warehouse/partition/nyc_taxis/data.parquet")

Py4JJavaError: An error occurred while calling o67.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.apa